In [1]:
!ls

Desktop    Downloads  localperl  Pictures  snap       Videos
Documents  gopath     Music	 Public    Templates


In [2]:
%cd ../../media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/protobuf/protobuf/

/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/protobuf/protobuf


In [6]:
%mkdir example2 
%cd example2/
!touch Dockerfile

/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/protobuf/protobuf/example2


## A second example of protobuf



okay so the previous try in installing and running the protobuf example in python was supper long and messy.

Now I want to try a more work relevant example with protobuf and ortools.


### first things first ...



still need a ubuntu, python, pip, etc.
but od we need both curl and wget?! need to fix this.

Looks like I can simplify the cmpiler installation for protobuf in python. Actually here seems I need to install compiler for grpc. 
DOn't how it is different than the protobuf.


Also here need to download the protobufs for the ORtools linear solver, etc.
```
rm -rf operations_research/linear_solver.proto
rm -rf operations_research/optional_boolean.proto

printf "\nGetting ortools/linear_solver/linear_solver.proto ....\n\n"
wget -q -P operations_research https://raw.githubusercontent.com/google/or-tools/v7.8/ortools/linear_solver/linear_solver.proto 

printf "Modifying ortools/linear_solver/linear_solver.proto with GO Language option ....\n\n"
sed '/^option/d' operations_research/linear_solver.proto | sed '/^import/d' | sed '32 i\option go_package="operations_research;operations_research";' | sed '35 i\import "optional_boolean.proto";'> operations_research/tmp
rm -f operations_research/linear_solver.proto
mv operations_research/tmp operations_research/linear_solver.proto 

printf "Getting ortools/util/optional_boolean.proto ....\n\n"
wget -q -P operations_research https://raw.githubusercontent.com/google/or-tools/v7.8/ortools/util/optional_boolean.proto 

printf "Modifyingortools/util/optional_boolean.proto with GO Language option ....\n\n"
sed '/^option/d' operations_research/optional_boolean.proto | sed '16 i\option go_package="operations_research;operations_research";' > operations_research/tmp
rm -f operations_research/optional_boolean.proto 
mv operations_research/tmp operations_research/optional_boolean.proto 

```

Then I will compile the protobuf using this command:

```
python3 -m grpc_tools.protoc -I operations_research --python_out=. --grpc_python_out=. operations_research/*.proto
```

So the overall Dockerfile looks like this:
```
FROM ubuntu

ADD ./install_python.sh .
RUN ./install_python.sh

ADD ./get_ortools_proto.sh .
RUN ./get_ortools_proto.sh 

ADD ./install_protobuf_compiler.sh .
RUN ./install_protobuf_compiler.sh

ADD ./mip.proto ./operations_research/
RUN python3 -m grpc_tools.protoc -I operations_research --python_out=. --grpc_python_out=. operations_research/*.proto


ADD ./mip_client.py  .
```

then finally I define my client code in python

```
import grpc 
import mip_pb2
import mip_pb2_grpc
import linear_solver_pb2

def create_model_request(model_request):
    model = model_request.model

    objective_coefficients = [10.0, 6.0, 4.0]
    variable_names = ["x1", "x2", "x3"]
    constraint_names = ["c1", "c2", "c3"]
    constraint_upbound = [100.0, 600.0, 300.0]
    constraint_coefficients = [[1.0, 1.0, 1.0], [10.0, 4.0, 5.0], [2.0, 2.0, 6.0]]

    # Set up variables
    num_var = len(variable_names)
    for i in range(num_var):
        variable = model.variable.add()
        variable.name = variable_names[i]
        variable.objective_coefficient = objective_coefficients[i]
        variable.lower_bound = 0.0 
        variable.is_integer = False  

    model.maximize = True

    # Set up constraints
    num_cons = len(constraint_names)
    for i in range(num_cons):
        constraint = model.constraint.add()
        constraint.name = constraint_names[i]
        constraint.upper_bound = constraint_upbound[i]
        # dealing with the constraint coefficient 
        for j in range(num_var):
            constraint.coefficient.append(constraint_coefficients[i][j])
            constraint.var_index.append(j)
    # Set up solver 
    model_request.solver_type  = linear_solver_pb2.MPModelRequest.SolverType.GLOP_LINEAR_PROGRAMMING

    return 

def read_solution(model_solution):
    print("The values of the variables:")
    variable_names = ["x1", "x2", "x3"]
    for j in range(3):
        print('Variable %s : %f' %(variable_names[j], model_solution.variable_value[j]))
    print("The objective value : %f" %(model_solution.objective_value))
    return 

def run(model_request):
    with grpc.insecure_channel('localhost:5050') as channel:
        stub = mip_pb2_grpc.MPServiceStub(channel)
        model_solution = stub.MPModel(model_request)
    read_solution(model_solution)
    return 

if __name__ == '__main__':
    model_request = linear_solver_pb2.MPModelRequest()
    create_model_request(model_request)
    run(model_request)

```


Odd Note: I obeserve some warning while the protobuf is compiled. Neeed to figure out why.

```
/usr/lib/python3.8/runpy.py:127: 
RuntimeWarning: 'grpc_tools.protoc' found in sys.modules after import of package 'grpc_tools', 
but prior to execution of 'grpc_tools.protoc';
 this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
```